# Handwritten digit recognition using MNIST database

This notebook gives step-by-step instruction to setup, train, evaluate and deploy neural network for handwritten digit recognition using ARM Machine Learning Platform.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
import model_manager as mm

In [2]:
print(tf.__version__)

2.0.0


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Develop a model

This section shows a an example model created using Tensorflow. This model is adapted from Tensorflow documentation.

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train the model

Train the model using the MNIST database.

In [5]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.2913 - accuracy: 0.9161
Epoch 2/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.1417 - accuracy: 0.9580
Epoch 3/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.1068 - accuracy: 0.9675
Epoch 4/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0871 - accuracy: 0.9739
Epoch 5/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0745 - accuracy: 0.9767


### Evaluate the model

The model is trained with 60K data points to get cose to 98% accuracy

In [6]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 0s - loss: 0.0347 - accuracy: 0.9778


[0.06853289482882247, 0.9778]

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Model Manager Library

The model manager library help manage models and their deployements. 

#### Manage models

In [8]:
mm.save_model(model, 'handwritten_digits')

=== save_model ===
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model-handwritten_digits-1.0.0/1/assets


List all the saved models.

In [16]:
mm.list_models()

[{'model_name': 'handwritten_digits', 'model_version': '1.0.0'}]

#### Manage deployments

The models are deployed in external docker containers running TFServing Engine. 

List all the deployments.

In [17]:
mm.list_deployments()

[]

Deploy a model.

In [18]:
mm.deploy_model('handwritten_digits')

{'model_name': 'handwritten_digits',
 'model_version': '1.0.0',
 'user': 'sk-arm'}

Use *test_prediction* API to test the deployed model.

In [19]:
in_data = x_test[0].flatten().tolist()
model_name = 'handwritten_digits'
model_version = '1.0.0'
resp = mm.test_prediction(in_data, model_name, model_version)
predicted_num = np.argmax(resp['predictions'][0])
print(predicted_num)

7


In [20]:
mm.list_deployments()

[{'model_name': 'handwritten_digits',
  'model_version': '1.0.0',
  'status': 'running'}]

Stop and removed the deployed model container.

In [21]:
mm.stop_deployment('handwritten_digits')

{'model_name': 'handwritten_digits', 'model_version': '1.0.0'}

In [22]:
mm.list_deployments()

[]